# [Day 19] 다섯 손가락의 합창: 멀티 제어와 최적화

어제 찾은 모터의 한계값을 이용해 오늘은 **다섯 손가락을 동시에** 제어해보겠습니다.
하지만 5개를 한꺼번에 계산하고 보내려면 컴퓨터가 바빠집니다.
렉(Lag)을 줄이기 위한 **최적화** 기술도 살짝 넣어보겠습니다.

## 1. 최적화의 핵심: "변한 것만 보내라"

손을 가만히 있는데도 계속 "90도.. 90도.. 90도.."라고 명령을 보내는 건 낭비입니다.
**이전 각도와 비교해서 차이가 클 때만** 명령을 보내면 훨씬 부드럽게 움직입니다.

In [ ]:
# 포트 선택 도구
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display

ports = serial.tools.list_ports.comports()
available_ports = [port.device for port in ports]
if not available_ports: available_ports = ['COM1']

port_dropdown = widgets.Dropdown(
    options=available_ports,
    value=available_ports[0],
    description='포트 선택:',
    disabled=False,
)
print("아두이노 포트를 선택하세요:")
display(port_dropdown)

In [ ]:
import cv2 as cv
import mediapipe as mp
import math
import serial
import time

# ----- 사용자 설정 (어제 찾은 값을 여기에 적으세요) -----# 예: 엄지는 0~90도만 움직여야 한다면 [0, 90]
# [엄지, 검지, 중지, 약지, 소지]
ROBOT_MIN = [0, 0, 0, 0, 0]
ROBOT_MAX = [180, 180, 180, 180, 180]

# 위에서 선택한 포트 사용
PORT = port_dropdown.value
try: ser = serial.Serial(PORT, 115200)except: ser = None

# --------------------------------------------------
# (기존 수학 함수들 생략 - 복사해서 쓰세요)def map_value(x, in_min, in_max, out_min, out_max):
    return (max(min(x, in_max), in_min) - in_min) * (out_max - out_min) / (in_max - in_min) + out_min
# --------------------------------------------------

# 이전 각도 저장용 리스트
prev_angles = [0, 0, 0, 0, 0]
last_send_time = 0

# 메인 루프 준비...
# (여기서는 코드 구조만 보여드립니다. 실제로는 전체 코드를 완성해야 합니다)
print("시스템 준비 완료. 손을 보여주세요!")

## 2. 전체 코드 구조

1. `for`문으로 5개 손가락을 돕니다.
2. 각 손가락마다 점 3개를 정해서 각도를 계산합니다.
3. `map_value`로 모터 각도를 구합니다.
4. `current_angles` 리스트에 담습니다.
5. `prev_angles`와 비교합니다. 차이가 5도 이상 나는 손가락이 하나라도 있으면 `전송`합니다.

### [도전 과제] 5개 손가락 연결하기

Day 17의 코드를 확장해서 5개 손가락을 모두 제어하는 코드를 완성해보세요.
각 손가락의 관절 번호는 다음과 같습니다.
- 엄지: 0-2-4
- 검지: 0-5-8
- 중지: 0-9-12
- 약지: 0-13-16
- 소지: 0-17-20